# Collecting auction data for Renoir

In [38]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
from tabulate import tabulate
import os
import numpy as np

#launch url
url = "http://www.askart.com/auction_records/Pierre_Auguste_Renoir/9000084/Pierre_Auguste_Renoir.aspx"


driver = webdriver.Chrome("/usr/local/bin/chromedriver")
driver.implicitly_wait(30)
driver.get(url)
soup_level1=BeautifulSoup(driver.page_source, 'lxml')


In [39]:
driver.find_element_by_id('MainPageContent_txtArtistName').send_keys("Auguste Renoir")

In [40]:
auto_complete = driver.find_elements_by_xpath(
        "//li[contains(@class, 'ui-menu-item')]")
auto_complete[0].click()

In [168]:
#driver.find_element_by_id('MainPageContent_searchButton').click()

In [41]:
driver.find_element_by_xpath('//span[contains(@class,"noUnderlineTOCitem") and contains(text(), "Auction Records")]').click()

In [42]:
selection = driver.find_elements_by_xpath('//div[@class="selectize-input items full has-options has-items"]')[0]
selection.click()

In [43]:
selection2 = driver.find_element_by_xpath('//div[@data-value="Painting"]').click()

In [44]:
Title_list=[]
Hammer_Price=[]
Low_Estimate_price=[]
High_Estimate_price=[]
Signature_list=[]
Size=[]
created_date=[]
Medium=[]
Auction_Lot=[]
Auction_House=[]
Auction_Date=[]

In [45]:
x=0
while x < 63:
    try:
        soup_level1=BeautifulSoup(driver.page_source, 'lxml')
        for link in soup_level1.find_all('div',attrs={"class":"PriceBox"}):
     #print(link.text)
                Title_list.append(link.contents[2].text) 
                Hammer_Price.append(link.contents[6])
                Low_Estimate_price.append(link.contents[11])
                High_Estimate_price.append(link.contents[14])
                if link.contents[21].string == "Signature:":
                    Signature_list.append(link.contents[22])
                else:
                    Signature_list.append("Unavailable")
               
                
                if link.contents[26].string == None:    
                    Size.append(link.contents[27])
                else:    
                    Size.append(link.contents[24])    
                     
                if  link.contents[29].string == "Created:":   
                    created_date.append(link.contents[30].string)#28
                elif (link.contents[29].string) == "Medium:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0":  
                    created_date.append(link.contents[27].string)
                else:  
                    if link.contents[31].string != None:
                        created_date.append(link.contents[31].string)
                    else:
                        created_date.append(link.contents[28].string)
               
                   
                if link.contents[33].string == "Medium:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0": 
                    Medium.append(link.contents[34].string)
                elif link.contents[33].string == "Auction Lot:":
                    Medium.append(link.contents[30].string)
                else:
                    Medium.append(link.contents[33].string)
                
                if link.contents[36].string =="Auction Lot:":  
                    Auction_Lot.append(link.contents[37].string)
                else:    
                    Auction_Lot.append(link.contents[34].string)
                
                if (link.contents[41].string) == None:
                    Auction_House.append(link.contents[42].string)
                else:
                    if ((link.contents[41].string)) == "Auction House:":
                            Auction_House.append(link.contents[42].string)
                    elif (link.contents[41].string) == "Auction Date:": 
                            Auction_House.append(link.contents[39].string)
                    else:
                            Auction_House.append(link.contents[41].string)
                if link.contents[44].string == None:
                    Auction_Date.append(link.contents[42].string)
                else:
                    if (link.contents[44].string) == "Auction Date:":
                            Auction_Date.append(link.contents[45].string)
                
                    else:
                            Auction_Date.append(link.contents[44].string)
       
        driver.find_element_by_xpath('//div[@class="pagination BiggerPaginator"]//a[@class="next"]').click()
        x+=1  
    except:
        break
        print("The scraping has came to an end") 
 
                 


In [46]:
vdict={'Title':Title_list,
'Hammer_Price':Hammer_Price,
'Low_Estimate_price':Low_Estimate_price,
'High_Estimate_price':High_Estimate_price,
'Signature':Signature_list,
'Size':Size,
'created_date':created_date,
'Medium':Medium,
'Auction_Lot':Auction_Lot,
'Auction_House':Auction_House,
'Auction_Date':Auction_Date}

In [47]:
df = pd.DataFrame(vdict,columns =['Title',"Hammer_Price","Low_Estimate_price","High_Estimate_price","Signature","Size","created_date","Medium","Auction_Lot","Auction_House","Auction_Date"]) 
df

,Title,Hammer_Price,Low_Estimate_price,High_Estimate_price,Signature,Size,created_date,Medium,Auction_Lot,Auction_House,Auction_Date
0,PAYSAGE À CAGNES,,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,c. 1910,Oil / Canvas,Oil / Canvas,,05/15/2019
1,JEUNE FILLE EN ROSE,,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,c. 1905,Oil / Canvas,Oil / Canvas,,05/15/2019
2,JEAN RENOIR EN MÉDAILLON,,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,1895,Oil / Canvas,Oil / Canvas,,05/15/2019
3,DEUX FEMMES,,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,c. 1903,Oil / Canvas,Oil / Canvas,,05/15/2019
4,SAULE AU BORD D'UNE MARE,,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,c. 1874,Oil / Canvas,Oil / Canvas,,05/15/2019
5,BAIGNEUSE AU LINGE,,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,c. 1915,Oil / Canvas,Oil / Canvas,,05/15/2019
6,Jeune fille en rose,,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,1900,Oil / Canvas,Oil / Canvas,,05/14/2019
7,Etude de jeune fille,,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,not given,Oil / Canvas,Oil / Canvas,,None
8,"Sucrier, gobelet, cuillère et couteau",,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,c.1910,Oil / Canvas,Oil / Canvas,,None
9,Vase de roses,,<b>Low Estimate:</b>,<b>High Estimate:</b>,Unavailable,<br/>,1906,Oil / Canvas,Oil / Canvas,,None


# Cleaning process for null, incomplete and inregular value

In [50]:
df=df.fillna("0")

In [51]:
houselist=[]

In [52]:
driver.find_element_by_xpath('//div[@class="pagination BiggerPaginator"]//a[@class="first"]').click()

In [53]:
x=0
while x < 63:
    try:
            soup_level1=BeautifulSoup(driver.page_source, 'lxml')
            for link in soup_level1.find_all('div',attrs={"class":"PriceBox"}):
                if link.contents[38].string != None:  
                    houselist.append(link.contents[38].string)
          
            driver.find_element_by_xpath('//div[@class="pagination BiggerPaginator"]//a[@class="next"]').click()
            x+=1  
    except:
            break
            print("The scraping has came to an end")

In [54]:
df["High_Estimate_price"]=df["High_Estimate_price"].replace('\(([^()]|())*\)', '',regex=True)
df["Low_Estimate_price"]=df["Low_Estimate_price"].replace('\(([^()]|())*\)', '',regex=True)
df["Title"]=df["Title"].replace('\(([^()]|())*\)', '',regex=True)

In [55]:
df["Low_Estimate_price"]=df["Low_Estimate_price"].str.replace('$', '')
df["Low_Estimate_price"]=df["Low_Estimate_price"].str.replace(',', '')
df["Low_Estimate_price"]=df["Low_Estimate_price"].str.replace('n/a', '0')
df["Low_Estimate_price"]=df["Low_Estimate_price"].str.replace('\xa0', '')


In [56]:
df["High_Estimate_price"]=df["High_Estimate_price"].str.replace('$', '')
df["High_Estimate_price"]=df["High_Estimate_price"].str.replace(',', '')
df["High_Estimate_price"]=df["High_Estimate_price"].str.replace('n/a', '0')
df["High_Estimate_price"]=df["High_Estimate_price"].str.replace('\xa0', '')

In [57]:
df["created_date"]=df["created_date"].str.replace('c.', '')
df["created_date"]=df["created_date"].str.replace('. ', '')
df["created_date"]=df["created_date"].str.replace('.', '')
df["Auction_House"]=df["Auction_House"].str.replace('\xa0', '')
df["Hammer_Price"]=df["Hammer_Price"].replace('\(([^()]|())*\)', '',regex=True)


In [58]:
df["Hammer_Price"]=df["Hammer_Price"].str.replace('$', '')
df["Hammer_Price"]=df["Hammer_Price"].str.replace(',', '')


In [78]:
for i in df.index:
    if df["Auction_House"][i] is not np.nan: 
        if df["Auction_House"][i][-1]==' ':
            df["Auction_Date"][i]=df["Auction_House"][i]

In [91]:
for i in df.index:
      for e in range(len(houselist)):
        if df["Auction_House"][i] !='':
            if df["Auction_House"][i][-1]==' ':
                    df["Auction_House"][i]=houselist[e]

In [71]:
import numpy as np
for i in df.index:
      if df["Auction_House"][i] =='':
            print(i)

59
176
738


In [77]:
df=df.drop([59,176,738])

In [49]:
df=df.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])#auction not yet coming 

In [102]:
df=df.drop(df[df["Auction_House"]=="\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0118"].index)

In [68]:
import collections

In [103]:
collections.Counter(df["Auction_House"])

Counter({'Accademia Fine Art': 1,
         'Anaf': 1,
         'Ansorena': 1,
         'Artcurial': 29,
         'Artcurial Deauville': 7,
         'Auction Date:': 3,
         'AuctionART-Remy Le Fur & Associes': 2,
         'Audap, Solanet & Godeau-Velliet': 1,
         'Auxerre Encheres': 1,
         'Baron Ribeyre & Associes': 2,
         'Bayeux Encheres': 2,
         'Beaussant Lefevre': 4,
         'Binoche et Giquello': 3,
         'Blanchet & Associes': 2,
         'Boisgirard-Antonini (Paris)': 2,
         'Bolland & Marotz': 1,
         'Bonhams Bond Street': 29,
         'Bonhams California': 1,
         'Bonhams New York': 5,
         'Butterscotch Auction Gallery': 3,
         'Cambi': 1,
         'Castells & Castells': 1,
         'Chaville Encheres': 1,
         'China Guardian Hong Kong Auctions, Co': 1,
         'Christian de Quay': 1,
         "Christie's Amsterdam": 1,
         "Christie's London, King Street": 215,
         "Christie's London, South Kensington": 7,

In [104]:
sold_status=[]

In [105]:
for i in df.index:
    if df["Hammer_Price"][i]=="-not sold-":
        sold_status.append("not sold")
    else:
        sold_status.append("sold")

In [106]:
df["sold_status"]=sold_status

In [107]:
df["Hammer_Price"]=df["Hammer_Price"].str.replace('-not sold-', '0')

In [108]:
for i in df.index:
     if df["Hammer_Price"][i] =='not communicated':
            df=df.drop([i])

In [ ]:
for i in df.index:
     if df["Hammer_Price"][i]=="awaiting result":
            df=df.drop([i])

In [109]:
df["Hammer_Price"]=df["Hammer_Price"].str.replace('\xa0', '')
df["created_date"]=df["created_date"].str.replace('\xa0', '')
df["Medium"]=df["Medium"].str.replace('\xa0', '')

In [110]:
df["Size"]=df["Size"].str.replace(')', '')
df["Size"]=df["Size"].str.replace('(', '')
df["Size"]=df["Size"].str.replace('cm', '')
df["Size"]=df["Size"].str.replace('\xa0', '')

In [112]:
for i in df.index:
    if df["Medium"][i]== None:
         df=df.drop([i])

In [113]:
import numpy as np
for i in df.index:
    if df["Size"][i] is np.nan:
        df=df.drop([i])

331
1483
1549


In [114]:
#df=df.drop([331,1483,1549])

In [115]:
Width=[]
Height=[]

In [116]:
for i in df["Size"].str.split("x"):
        Width.append(i[0])
        Height.append(i[1])  

In [117]:
df["height"]=Height
df["width"]=Width
df["width"]=df["width"].astype(float)
df["height"]=df["height"].astype(float)

In [118]:
df["Signature"]=df["Signature"].str.replace("\xa0","")
df["Auction_Date"]=df["Auction_Date"].str.replace("\xa0","")
df["Auction_Lot"]=df["Auction_Lot"].str.replace("\xa0","")

In [119]:
df=df.drop(["Size"],axis=1)

In [121]:
#save data
df.to_csv("Renoir_auc.csv", index=False,encoding="utf-8")#  while do not open this csv in folder directly!!!